<a href="https://colab.research.google.com/github/hollimey/capstone-application/blob/main/OceanWeatherApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Get User Location Input

In [ ]:
# get user location input (U.S. only)
user_location = input("Enter a U.S. location to get weather information (ex. Seattle, WA): ")

Enter a U.S. location to get weather information (ex. Seattle, WA): seattle, wa


#### Convert to Coords

In [ ]:
# use geocoding library to convert city name into lat/lon coordinates

%pip install geopy

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable

geolocator = Nominatim(user_agent="weather_app")

try:
    user_location_geocoded = geolocator.geocode(user_location)
    if user_location_geocoded:
        latitude = user_location_geocoded.latitude
        longitude = user_location_geocoded.longitude
        print(f"Coordinates: {latitude}, {longitude}")
    else:
        print(f"Location '{user_location}' not found.")
        latitude = None
        longitude = None
except (GeocoderTimedOut, GeocoderUnavailable) as e:
    print(f"Geocoding service error: {e}")
    latitude = None
    longitude = None

Coordinates: 47.6038321, -122.330062


### NOAA Weather Data

In [ ]:
# fetch The National Weather Service (NWS) data from API

import requests
import json
from datetime import datetime

def fetch_weather_data(latitude, longitude):
    point_url = f"https://api.weather.gov/points/{latitude},{longitude}"
    try:
        response = requests.get(point_url)
        response.raise_for_status()  # http error for bad responses
        point_data = response.json()

        forecast_url = point_data['properties']['forecast']
        hourly_forecast_url = point_data['properties']['forecastHourly']
        gridpoints_url = f"https://api.weather.gov/gridpoints/{point_data['properties']['gridId']}/{point_data['properties']['gridX']},{point_data['properties']['gridY']}/forecast"

        # fetch data
        forecast_response = requests.get(forecast_url)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()
        hourly_forecast_response = requests.get(hourly_forecast_url)
        hourly_forecast_response.raise_for_status()
        hourly_forecast_data = hourly_forecast_response.json()
        gridpoints_response = requests.get(gridpoints_url)
        gridpoints_response.raise_for_status()
        gridpoints_data = gridpoints_response.json()

        # return dictionary
        return {
            "point_data": point_data,
            "forecast": forecast_data,
            "hourly_forecast": hourly_forecast_data,
            "gridpoints": gridpoints_data
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching atmospheric data: {e}")
        return None

weather_data_result = fetch_weather_data(latitude, longitude)

# determine the state to fetch alerts
state_code = None
if weather_data_result and 'point_data' in weather_data_result and \
   'properties' in weather_data_result['point_data'] and \
   'relativeLocation' in weather_data_result['point_data']['properties'] and \
   'properties' in weather_data_result['point_data']['properties']['relativeLocation'] and \
   'state' in weather_data_result['point_data']['properties']['relativeLocation']['properties']:
    state_code = weather_data_result['point_data']['properties']['relativeLocation']['properties']['state']
    print(f"Determined state for alerts: {state_code}")
else:
    print("Could not determine state for fetching alerts.")

# initialize
parsed_alerts = []
alerts_url = None

# fetch and parse alerts if state_code is available
if state_code:
    alerts_url = f"https://api.weather.gov/alerts/active?area={state_code}"

    try:
        alerts_response = requests.get(alerts_url)
        if alerts_response.status_code == 200:

            # parse json response
            weather_alerts_data = alerts_response.json()
            if weather_alerts_data and 'features' in weather_alerts_data:
                for feature in weather_alerts_data['features']:
                    if 'properties' in feature:
                        properties = feature['properties']
                        alert_info = {
                            'event': properties.get('event'),
                            'headline': properties.get('headline'),
                            'description': properties.get('description'),
                            'effective': properties.get('effective'),
                            'expires': properties.get('expires')
                        }
                        parsed_alerts.append(alert_info)
            print(f"Parsed {len(parsed_alerts)} active weather alerts.")
        else:
            print(f"Error fetching weather alerts: Status code {alerts_response.status_code}")
            print(f"Response content:\n{alerts_response.text}")
            weather_alerts_data = None
    except requests.exceptions.RequestException as e:
        print(f"Error during weather alerts request: {e}")
        weather_alerts_data = None
else:
    print("State code/abbreviation not available. Cannot fetch alerts.")
    weather_alerts_data = None

# save to json file
if weather_data_result:
    # remove useless "@context" key information:
    # from point_data
    if 'point_data' in weather_data_result and '@context' in weather_data_result['point_data']:
        del weather_data_result['point_data']['@context']
    # from forecast
    if 'forecast' in weather_data_result and '@context' in weather_data_result['forecast']:
        del weather_data_result['forecast']['@context']
    # from hourly_forecast
    if 'hourly_forecast' in weather_data_result and '@context' in weather_data_result['hourly_forecast']:
        del weather_data_result['hourly_forecast']['@context']
    # from gridpoints
    if 'gridpoints' in weather_data_result and '@context' in weather_data_result['gridpoints']:
        del weather_data_result['gridpoints']['@context']

    # remove empty "detailedForecast" information:
    # from hourly_forecast periods
    if 'hourly_forecast' in weather_data_result and 'properties' in weather_data_result['hourly_forecast'] and 'periods' in weather_data_result['hourly_forecast']['properties']:
        for period in weather_data_result['hourly_forecast']['properties']['periods']:
            if 'detailedForecast' in period:
                del period['detailedForecast']

    # remove empty "name" information:
    # from hourly_forecast periods
    if 'hourly_forecast' in weather_data_result and 'properties' in weather_data_result['hourly_forecast'] and 'periods' in weather_data_result['hourly_forecast']['properties']:
        for period in weather_data_result['hourly_forecast']['properties']['periods']:
            if 'name' in period:
                del period['name']

    # remove useless "icon" information:
    # from hourly_forecast periods
    if 'hourly_forecast' in weather_data_result and 'properties' in weather_data_result['hourly_forecast'] and 'periods' in weather_data_result['hourly_forecast']['properties']:
        for period in weather_data_result['hourly_forecast']['properties']['periods']:
            if 'icon' in period:
                del period['icon']

    # remove empty "temperatureTrend" information:
    # from gridpoints periods
    if 'gridpoints' in weather_data_result and 'properties' in weather_data_result['gridpoints'] and 'periods' in weather_data_result['gridpoints']['properties']:
        for period in weather_data_result['gridpoints']['properties']['periods']:
            if 'temperatureTrend' in period:
                del period['temperatureTrend']
    # from hourly_forecast periods
    if 'hourly_forecast' in weather_data_result and 'properties' in weather_data_result['hourly_forecast'] and 'periods' in weather_data_result['hourly_forecast']['properties']:
        for period in weather_data_result['hourly_forecast']['properties']['periods']:
            if 'temperatureTrend' in period:
                del period['temperatureTrend']
    # from forecast periods
    if 'forecast' in weather_data_result and 'properties' in weather_data_result['forecast'] and 'periods' in weather_data_result['forecast']['properties']:
        for period in weather_data_result['forecast']['properties']['periods']:
            if 'temperatureTrend' in period:
                del period['temperatureTrend']

    # structure json file
    weather_output = {
        "user_location": {
            "latitude": latitude,
            "longitude": longitude,
            "date": datetime.now().strftime('%Y-%m-%d')
        },
        "atmospheric_data": weather_data_result,
        "weather_alerts": parsed_alerts,
        "alerts_source_url": alerts_url
    }
    weather_file = "user_data_noaa_atmos.json"
    try:
        with open(weather_file, 'w') as f:
            json.dump(weather_output, f, indent=4)
        print(f"\nAtmospheric data and alerts saved to {weather_file}")
    except Exception as e:
        print(f"Error saving atmospheric data and alerts to JSON: {e}")
else:
    print("Atmospheric data has not been fetched yet.")

Determined state for alerts: WA
Parsed 5 active weather alerts.

Atmospheric data and alerts saved to user_data_noaa_atmos.json


### EPA Air Quality Data

In [ ]:
# fetch AQ data from EPA API

import requests
import json
from datetime import datetime

# note: consider adding a secure method for handling API keys later
airnow_api_key = "17C7530F-6ED9-40D5-8DA3-A8328CB8F1B0"
airnow_url = f"https://www.airnowapi.org/aq/observation/latLong/current/?format=application/json&latitude={latitude}&longitude={longitude}&distance=25&API_KEY={airnow_api_key}"

try:
    response = requests.get(airnow_url)
    response.raise_for_status() # raise http error for bad responses
    aq_data = response.json()
except requests.exceptions.RequestException as e:
    print(f"Error fetching air quality data: {e}")
    aq_data = None

# save to json file
if aq_data:
    aq_file = "user_data_epa_aq.json"

    output_data = {
        "user_location": {
            "latitude": latitude,
            "longitude": longitude,
            "date": datetime.now().strftime('%Y-%m-%d')
        },
        "air_quality_data": aq_data # include fetched AQ data under a new key
    }

    try:
        # write to json file
        with open(aq_file, 'w') as f:
            json.dump(output_data, f, indent=4)
        print(f"Air quality data saved to {aq_file}")
    except Exception as e:
        print(f"Error saving air quality data to JSON: {e}")
else:
    print("Air quality data not available to save.")

Air quality data saved to user_data_epa_aq.json


### NOAA Buoy Data

In [ ]:
# fetch and parse ocean buoy data

import requests
import pandas as pd
from io import StringIO
import json

buoy_url = "https://www.ndbc.noaa.gov/data/latest_obs/latest_obs.txt"

try:
  response = requests.get(buoy_url)
  response.raise_for_status()
  text_content = response.text

  # parse the text content into a pandas df
  data_lines = text_content.strip().split('\n')
  header_line_index = -1
  for i, line in enumerate(data_lines):
      if line.startswith('#'):
          header_line_index = i
          break

  if header_line_index != -1:
      buoy_dataframe = pd.read_csv(StringIO('\n'.join(data_lines[header_line_index:])), sep='\s+', skiprows=[1]) # skips units row

      # save to json file
      ocean_buoy_file = "dataset_noaa_buoy.json"
      try:
          # convert df to a list of dictionaries for json serialization
          buoy_data_list = buoy_dataframe.to_dict(orient='records')

          # structure json file
          output_data = {
              "buoy_observations": buoy_data_list
          }

          with open(ocean_buoy_file, 'w') as f:
              json.dump(output_data, f, indent=4)
          print(f"Buoy data saved to {ocean_buoy_file}")
      except Exception as e:
          print(f"Error saving NOAA buoy data to JSON: {e}")
  else:
      print("Could not find header line in NOAA buoy data.")
      buoy_dataframe = None
except requests.exceptions.RequestException as e:
    print(f"Error fetching NOAA buoy data: {e}")
    buoy_dataframe = None
except Exception as e:
    print(f"Error parsing NOAA buoy data: {e}")
    buoy_dataframe = None

Buoy data saved to dataset_noaa_buoy.json


#### Nearest Stations


In [ ]:
from geopy.distance import geodesic
import pandas as pd
import json
from datetime import datetime

if latitude is None or longitude is None:
    print("User location could not be determined.")
    buoy_dataframe = None

# calculate distance between user and each buoy
if buoy_dataframe is not None:
    def calculate_distance(row):
        station_lat = pd.to_numeric(row['LAT'], errors='coerce')
        station_lon = pd.to_numeric(row['LON'], errors='coerce')

        if pd.isna(station_lat) or pd.isna(station_lon):
            return float('inf')
        try:
            user_coords = (latitude, longitude)
            station_coords = (station_lat, station_lon)
            return geodesic(user_coords, station_coords).miles
        except ValueError:
            return float('inf')
    buoy_dataframe['Distance'] = buoy_dataframe.apply(calculate_distance, axis=1)
    nearest_stations_df = buoy_dataframe.sort_values(by='Distance').reset_index(drop=True) # sort by distance

    requested_parameters = {
        'wind_direction': 'WDIR',
        'wind_speed': 'WSPD',
        'wind_gust': 'GST',
        'wave_height': 'WVHT',
        'dominant_wave_period': 'DPD',
        'avg_wave_period': 'APD',
        'mean_wave_direction': 'MWD',
        'atmos_pressure': 'PRES',
        'pressure_tendency': 'PTDY',
        'air_temp': 'ATMP',
        'water_temp': 'WTMP',
        'dewpoint_temp': 'DEWP',
        'visibility': 'VIS',
        'tide': 'TIDE'
    }

    extracted_data = {}
    parameters_found = {
        param: False for param in requested_parameters.keys()
        }
    searched_stations = []
    search_radius = 80 # define the search radius

    # iterate through stations starting from the nearest
    for index, station in nearest_stations_df.iterrows():
        station_id = station['#STN']
        station_distance = round(station['Distance'], 2) # round distance

        # check if the station is within radius
        if station_distance <= search_radius:
            searched_stations.append({'id': station_id, 'distance': station_distance})
            all_found_for_this_station = True

            for param_name, col_name in requested_parameters.items():
                if not parameters_found[param_name]:
                    param_value = station.get(col_name)

                    if pd.notna(param_value) and str(param_value).strip().upper() != 'MM':
                        measurement_unit = parameter_metadata.get(col_name, "").split('(')[-1].replace(')', '')

                        try:
                            year = int(station.get('YYYY'))
                            month = int(station.get('MM'))
                            day = int(station.get('DD'))
                            hour = int(station.get('hh'))
                            minute = int(station.get('mm'))
                            observation_time_lst = datetime(year, month, day, hour, minute).strftime('%Y-%m-%d %H:%M:%S')
                        except (ValueError, TypeError):
                            observation_time_lst = "N/A"

                        extracted_data[param_name] = {
                            'value': param_value,
                            'unit': measurement_unit.strip(),
                            'station_id': station_id,
                            'station_latitude': station.get('LAT'),
                            'station_longitude': station.get('LON'),
                            'time_LST': observation_time_lst,
                            'distance_miles': station_distance
                            }
                        parameters_found[param_name] = True
                    else:
                       all_found_for_for_this_station = False
            if all(parameters_found.values()):
                print("All requested parameters found within the 80-mile radius.")
                break # exits loop when all parameters are found
        else:
            break # stops when exceeds radius

    # count the number of parameters found
    num_parameters_found = sum(parameters_found.values())
    total_parameters = len(requested_parameters)
    print(f"Found data for {num_parameters_found} out of {total_parameters} parameters within the {search_radius}-mile radius.")

    # identify missing parameters
    missing_params = [param for param, found in parameters_found.items() if not found]

    # structure json file
    final_output = {
        "user_location": {
            "latitude": latitude,
            "longitude": longitude,
            "date": datetime.now().strftime('%Y-%m-%d')
        },
        "extracted_buoy_data": extracted_data,
        "search_radius_miles": search_radius,
        "missing_parameters": missing_params,
        "searched_stations_within_radius": searched_stations
    }

    # save json file
    if final_output:
        output_filename = "user_data_noaa_buoy.json"
        try:
            with open(output_filename, 'w') as f:
                json.dump(final_output, f, indent=4)
            print(f"\nNearest buoy data saved to {output_filename}")
        except Exception as e:
            print(f"Error saving nearest buoy data to JSON: {e}")
    else:
        print("No nearest buoy data available to save.")
else:
    print("Buoy data is not available to extract data.")

Found data for 12 out of 14 parameters within the 80-mile radius.

Nearest buoy data saved to user_data_noaa_buoy.json


### CO-OPS Stations

In [ ]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup

stations_json_url = "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations.json"
stations_xml_url = "https://opendap.co-ops.nos.noaa.gov/stations/stationsXML.jsp"

try:
    # stations from JSON source
    response = requests.get(stations_json_url)
    response.raise_for_status()
    stations_data = response.json()

    if stations_data and 'stations' in stations_data and isinstance(stations_data['stations'], list):
        stations_df = pd.DataFrame(stations_data['stations'])
        print(f"Fetched {len(stations_df)} stations from: {stations_json_url}")

        selected_columns_df = stations_df[['id', 'name', 'state', 'lng', 'lat']].copy()
        selected_columns_df = selected_columns_df.rename(columns={
            'id': 'station_id',
            'name': 'Station Name',
            'state': 'State',
            'lng': 'Longitude',
            'lat': 'Latitude'
        })

        selected_columns_df['City, State'] = selected_columns_df['Station Name'] + ', ' + selected_columns_df['State']
        formatted_stations_df = selected_columns_df[['station_id', 'City, State', 'Longitude', 'Latitude']]
    else:
        print("JSON data does not contain a list under the key 'stations'.")
        stations_df = pd.DataFrame()
        formatted_stations_df = pd.DataFrame()

    # stations from XML source
    xml_response = requests.get(stations_xml_url)
    xml_response.raise_for_status()
    soup = BeautifulSoup(xml_response.content, "xml")

    xml_stations = []
    for st in soup.find_all("station"):
        xml_stations.append({
            "station_id": st.get("ID"),
            "Owner": st.get("Owner"),
            "Type": st.get("Type"),
            "Status": st.get("Status")
        })

    xml_df = pd.DataFrame(xml_stations)
    print(f"Fetched {len(xml_df)} stations from: {stations_xml_url}")

    # merge data
    merged_nearby_stations_df = pd.merge(formatted_stations_df, xml_df, on="station_id", how="left")

    # remove empty 'Owner', 'Type', and 'Status' columns
    columns_to_drop = ['Owner', 'Type', 'Status']
    merged_nearby_stations_df = merged_nearby_stations_df.drop(columns=columns_to_drop, errors='ignore') # if columns don't exist

    # save json file
    output_filename = "dataset_noaa_coops.json"
    try:
        merged_data_json = merged_nearby_stations_df.to_dict(orient='records')

        # structure json file
        output_data = {
            "datasets": {
                "sources": [stations_json_url, stations_xml_url],
                "total_stations_listed": len(merged_nearby_stations_df) if not merged_nearby_stations_df.empty else 0,
                "description": {
                  "about": "Maintained by the National Oceanic and Atmospheric Administration's (NOAA), Center for Operational Oceanographic Products and Services (CO-OPS). Provided is a list of active National Water Level Observation Network (NWLON) stations with some sensor configurations. More information can be found through https://opendap.co-ops.nos.noaa.gov/stations/index.jsp"
                },
            "station_list": merged_data_json
            },
        }

        with open(output_filename, 'w') as f:
            json.dump(output_data, f, indent=4)
        print(f"\nCO-OPS stations saved to {output_filename}")
    except Exception as e:
        print(f"Error saving merged CO-OPS station data to JSON: {e}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    stations_df = pd.DataFrame()
    formatted_stations_df = pd.DataFrame()
    merged_nearby_stations_df = pd.DataFrame()
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    stations_df = pd.DataFrame()
    formatted_stations_df = pd.DataFrame()
    merged_nearby_stations_df = pd.DataFrame()

Fetched 301 stations from: https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations.json
Fetched 381 stations from: https://opendap.co-ops.nos.noaa.gov/stations/stationsXML.jsp

CO-OPS stations saved to dataset_noaa_coops.json


#### Nearest Stations

In [ ]:
from geopy.distance import geodesic
import pandas as pd
import json
from datetime import datetime

# confirm user location
if 'latitude' not in globals() or latitude is None or 'longitude' not in globals() or longitude is None:
    print("User location (latitude or longitude) not available. Cannot filter stations by proximity.")
elif 'formatted_stations_df' not in globals() or formatted_stations_df.empty:
    print("Formatted stations data is not available. Cannot filter stations.")
else:
    user_coords = (latitude, longitude)
    proximity_miles = 80

    # calculate distance from user to each station
    stations_for_distance_calc = formatted_stations_df.copy()
    stations_for_distance_calc['Latitude'] = pd.to_numeric(stations_for_distance_calc['Latitude'], errors='coerce')
    stations_for_distance_calc['Longitude'] = pd.to_numeric(stations_for_distance_calc['Longitude'], errors='coerce')

    stations_for_distance_calc.dropna(subset=['Latitude', 'Longitude'], inplace=True) # drop lat/lon rows that could not be converted to numeric

    def calculate_distance_miles(row):
        station_coords = (row['Latitude'], row['Longitude'])
        try:
            return geodesic(user_coords, station_coords).miles
        except ValueError:
            return float('inf') # handles invalid coords

    stations_for_distance_calc['distance_miles'] = stations_for_distance_calc.apply(calculate_distance_miles, axis=1)

    # filter for stations within the specified proximity
    merged_nearby_stations_df = stations_for_distance_calc[stations_for_distance_calc['distance_miles'] <= proximity_miles].sort_values(by='distance_miles')

    if not merged_nearby_stations_df.empty:
        merged_nearby_stations_df['distance_miles'] = merged_nearby_stations_df['distance_miles'].round(2)

        # save json file
        output_filename = "user_data_noaa_coops_stations.json"
        try:
            nearby_stations_json = merged_nearby_stations_df.to_dict(orient='records')

            # structure json file
            output_data = {
                "user_location": {
                    "latitude": latitude,
                    "longitude": longitude,
                    "date": datetime.now().strftime('%Y-%m-%d')
                },
                "coops_stations": {
                    "source_url": stations_json_url,
                    "within_proximity_miles": proximity_miles,
                    "total_stations_within_proximity": len(merged_nearby_stations_df),
                    "nearby": {
                        "stations": merged_nearby_stations_df.to_dict(orient='records')
                    },
                },
            }

            with open(output_filename, 'w') as f:
                json.dump(output_data, f, indent=4)
            print(f"Nearby CO-OPS stations saved to {output_filename}")
        except Exception as e:
            print(f"Error saving nearby CO-OPS station data to JSON: {e}")
    else:
        print("No CO-OPS stations found within the specified proximity.")

Nearby CO-OPS stations saved to user_data_noaa_coops_stations.json


#### Tide Predictions

In [ ]:
# fetch tide predictions from CO-OPS API

import requests
import json
from datetime import datetime, timedelta
import pandas as pd

if 'merged_nearby_stations_df' not in globals() or merged_nearby_stations_df.empty:
    print("Nearby CO-OPS stations are unavailable at this time.")
else:
    today = datetime.now() # date range for yesterday and today
    yesterday = today - timedelta(days=1)

    begin_date_str = yesterday.strftime('%Y%m%d')
    end_date_str = today.strftime('%Y%m%d') # format string YYYYMMDD

    tide_predictions_url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"

    about_tides = "Tides refer to the rising and falling of the sea. Which is caused by the gravitational pull of the moon and the sun. Tides are very long-period waves that move through the ocean and progress toward the coastlines where they appear as the regular rise and fall of the sea surface. Tide predictions provide the times and heights for the astronomical tides. Predictions are based on the analysis of data collected at coastal locations maintained by the National Oceanic and Atmospheric Administration's (NOAA), Center for Operational Oceanographic Products and Services (CO-OPS). CO-OPS maintains the National Water Level Observation Network (NWLON), an observation network with more than 200 permanent water level stations on the coasts and Great Lakes.This system allows NOAA to provide the official tidal predictions for the nation. Accurate water level data is critical for safe and efficient marine navigation and for the protection of infrastructure along the coast.Harmonic stations are locations with enough long-term tide data to establish harmonic constants and tidal datums. Predictions for these stations are based solely on the analysis of that data. Because predictions at these locations are based on harmonic constants, predictions can be generated for any interval, and can be adjusted to different tidal datums. CO-OPS has preselected the most common intervals (hourly, 15- and 6-minute) for data queries. Learn more through https://tidesandcurrents.noaa.gov/water_level_info.html or https://tidesandcurrents.noaa.gov/education/tech-assist/training/user-guides/assets/pdfs/Tide_Predictions_User_Guide_v4.pdf"

    # initialize variables
    tide_predictions_result = None
    found_data = False
    nearest_station_info = None

    # iterate through nearby stations to find one with tide predictions
    for index, station in merged_nearby_stations_df.iterrows():
        nearest_station_id = station['station_id']

        # API request parameters
        params = {
            'product': 'predictions',
            'application': 'NOS.COOPS.TAC.WL',
            'begin_date': begin_date_str,
            'end_date': end_date_str,
            'datum': 'MLLW',
            'station': nearest_station_id,
            'time_zone': 'lst_ldt', # local time with daylight saving
            'units': 'english',
            'interval': '', # leave interval empty for all predictions
            'format': 'json'
        }

        try:
            response = requests.get(tide_predictions_url, params=params)
            if response.status_code == 200:

                 # parse the JSON response
                 tide_predictions_data = response.json()
                 tide_predictions_result = tide_predictions_data

                 # check if data is available in the response
                 if tide_predictions_result and 'predictions' in tide_predictions_result and tide_predictions_result['predictions']:

                    nearest_station_info = {
                        "station_id": nearest_station_id,
                        "station_name": station.get('City, State', 'N/A'),
                        "distance_miles": round(station.get('distance_miles', float('inf')), 2),
                        "latitude": station.get('Latitude', 'N/A'),
                        "longitude": station.get('Longitude', 'N/A'),
                        "source_url": response.url
                    }
                    found_data = True
                    break
            else:
                print(f"Error fetching tide predictions from station {nearest_station_id}: Status code {response.status_code}")
                print(f"Response content:\n{response.text}")
                tide_predictions_result = None # store None if fetch fails
        except requests.exceptions.RequestException as e:
            print(f"Error during tide prediction request for station {nearest_station_id}: {e}")
            tide_predictions_result = None # store None if request fails

    # check if data was found after iterating through stations
    if found_data:
        reformatted_predictions = []
        if tide_predictions_result and 'predictions' in tide_predictions_result:
            predictions_list = tide_predictions_result['predictions']

            if predictions_list:

                # save json file
                output_filename = "user_data_noaa_coops_tide.json"

                try:
                    reformatted_predictions = []
                    for prediction in predictions_list:
                        reformatted_predictions.append({
                            "time_LST": prediction.get('t'),
                            "water_level_ft": prediction.get('v')
                        })

                    # structure json file
                    output_data = {
                        "user_location": {
                            "latitude": latitude,
                            "longitude": longitude,
                            "date": datetime.now().strftime('%Y-%m-%d')
                        },
                        "tide_predictions": {
                            "background": {
                                "about": about_tides,
                                "nearest_station": {
                                  "station_info": nearest_station_info,
                                  "data": {
                                    "predictions": reformatted_predictions
                                },
                              },
                            },
                          },
                        }

                    with open(output_filename, 'w') as f:
                        json.dump(output_data, f, indent=4, ensure_ascii=False)
                    print(f"Tide prediction data saved to {output_filename}")
                except Exception as e:
                    print(f"Error saving tide prediction data to JSON: {e}")
    else:
        print("No tide predictions found for nearby CO-OPS stations.")

Tide prediction data saved to user_data_noaa_coops_tide.json


#### Water Level

In [ ]:
# fetch water level observations from CO-OPS API

import requests
import json
from datetime import datetime, timedelta
import pandas as pd

if 'merged_nearby_stations_df' not in globals() or merged_nearby_stations_df.empty:
    print("Nearby CO-OPS stations data is not available. Cannot fetch water level data.")
else:
    end_date = datetime.now()
    begin_date = end_date - timedelta(days=1) # last 24 hours date range

    begin_date_str = begin_date.strftime('%Y%m%d') # string format YYYYMMDD
    end_date_str = end_date.strftime('%Y%m%d')

    water_level_url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"

    # initialize variables
    water_level_result = None
    found_data = False

    # iterate through nearby stations to find one with water level data
    for index, station in merged_nearby_stations_df.iterrows():
        nearest_station_id = station['station_id']

        # API request parameters
        params = {
            'product': 'water_level',
            'application': 'NOS.COOPS.TAC.WL',
            'begin_date': begin_date_str,
            'end_date': end_date_str,
            'datum': 'MLLW',
            'station': nearest_station_id,
            'time_zone': 'lst', # using local standard time
            'units': 'english',
            'format': 'json',
            'interval': '6', # updates data at 6-minute intervals
        }

        try:
            response = requests.get(water_level_url, params=params)
            if response.status_code == 200:

                # parse the json response
                water_level_data = response.json()
                water_level_result = water_level_data

                # check if data is available in the response
                if water_level_result and 'data' in water_level_result and water_level_result['data']:

                    # reformat keys
                    reformatted_data = []
                    for entry in water_level_result['data']:
                        reformatted_entry = {
                            "time_LST": entry.get('t'),
                            "water_level_ft": entry.get('v'),
                            "sigma": entry.get('s'),
                            "quality_flags": entry.get('f'),
                            "quality_control_flag": entry.get('q')
                        }
                        reformatted_data.append(reformatted_entry)

                    # structure json file
                    output_data = {
                        "user_location": {
                            "latitude": latitude,
                            "longitude": longitude,
                            "date": datetime.now().strftime('%Y-%m-%d')
                        },
                        "water_level": {
                            "station_info": {
                                "station_id": nearest_station_id,
                                "station_name": station.get('City, State', 'N/A'),
                                "distance_miles": round(station.get('distance_miles', float('inf')), 2),
                                "latitude": station.get('Latitude', 'N/A'),
                                "longitude": station.get('Longitude', 'N/A'),
                                "source_url": response.url
                            },
                              "data": reformatted_data
                        },
                    }

                    # save json file
                    output_filename = "user_data_noaa_coops_waterlevel.json"
                    try:
                        with open(output_filename, 'w') as f:
                            json.dump(output_data, f, indent=4)
                        print(f"Water level observation data saved to {output_filename}")
                    except Exception as e:
                        print(f"Error saving water level observation data to JSON: {e}")

                    found_data = True
                    break
                else:
                     print(f"No water level data available for station {nearest_station_id}.")
            else:
                print(f"Error fetching water level data: Status code {response.status_code}")
                print(f"Response content:\n{response.text}")
                water_level_result = None # store None if fetch fails
        except requests.exceptions.RequestException as e:
            print(f"Error during water level data request for station {nearest_station_id}: {e}")
            water_level_result = None # store None if request fails
    if not found_data:
        print("Could not find water level observations from any nearby CO-OPS station.")

Water level observation data saved to user_data_noaa_coops_waterlevel.json


#### Air Temp

In [ ]:
# fetch air temperature data from CO-OPS API

import requests
import json
from datetime import datetime, timedelta
import pandas as pd

if 'merged_nearby_stations_df' not in globals() or merged_nearby_stations_df.empty:
    print("Nearby CO-OPS stations data is not available. Cannot fetch air temperature data.")
else:
    today = datetime.now()
    begin_date_str = today.strftime('%Y%m%d')
    end_date_str = today.strftime('%Y%m%d')

    air_temp_url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"

    # initialize variables
    air_temp_result = None
    found_data = False
    nearest_station_info = None

    # iterate through nearby stations to find one with air temp data
    for index, station in merged_nearby_stations_df.iterrows():
        nearest_station_id = station['station_id']

        # API request parameters
        params = {
            'product': 'air_temperature',
            'application': 'NOS.COOPS.TAC.MET',
            'begin_date': begin_date_str,
            'end_date': end_date_str,
            'station': nearest_station_id,
            'units': 'metric',
            'time_zone': 'lst',
            'format': 'json',
        }

        try:
            response = requests.get(air_temp_url, params=params)

            if response.status_code == 200:
                air_temp_data = response.json()
                if air_temp_data and 'data' in air_temp_data and air_temp_data['data']:
                    air_temp_result = air_temp_data

                    nearest_station_info = {
                        "station_id": nearest_station_id,
                        "station_name": station.get('City, State', 'N/A'),
                        "distance_miles": round(station.get('distance_miles', float('inf')), 2),
                        "latitude": station.get('Latitude', 'N/A'),
                        "longitude": station.get('Longitude', 'N/A'),
                        "source_url": response.url
                    }
                    found_data = True
                    break
            else:
                print(f"Error fetching air temperature data from station {nearest_station_id}: Status code {response.status_code}")
                print(f"Response content:\n{response.text}")
        except requests.exceptions.RequestException as e:
            print(f"Error during air temperature data request for station {nearest_station_id}: {e}")

    if found_data:
        if air_temp_result and 'data' in air_temp_result:
            air_temp_list = air_temp_result['data']
            if air_temp_list:

                # save json file
                output_filename = "user_data_noaa_coops_airtemp.json"
                try:
                    reformatted_data = []
                    for entry in air_temp_list:
                        temp_celsius = float(entry.get('v')) if entry.get('v') is not None else None
                        temp_fahrenheit = (temp_celsius * 9/5) + 32 if temp_celsius is not None else None

                        reformatted_data.append({
                            "time_LST": entry.get('t'),
                            "temp_fahrenheit": temp_fahrenheit,
                            "quality_flags": entry.get('f')
                        })

                    # structure json file
                    output_data = {
                        "user_location": {
                            "latitude": latitude,
                            "longitude": longitude,
                            "date": datetime.now().strftime('%Y-%m-%d')
                        },
                        "air_temperature": {
                            "station_info": nearest_station_info,
                            "data": reformatted_data
                        },
                    }

                    with open(output_filename, 'w') as f:
                        json.dump(output_data, f, indent=4)
                    print(f"Air temperature data saved to {output_filename}")
                except Exception as e:
                    print(f"Error saving air temperature data to JSON: {e}")
    else:
        print("No air temperatures found for nearby CO-OPS stations.")

Air temperature data saved to user_data_noaa_coops_airtemp.json


#### Water Temp

In [ ]:
# fetch water temperature data from CO-OPS API

import requests
import json
from datetime import datetime, timedelta
import pandas as pd

if 'merged_nearby_stations_df' not in globals() or merged_nearby_stations_df.empty:
    print("Nearby station for CO-OPS data is not available. Cannot fetch water temperature data.")
else:
    today = datetime.now()
    begin_date_str = today.strftime('%Y%m%d')
    end_date_str = today.strftime('%Y%m%d')

    water_temp_url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"

    # initialize variables
    water_temp_result = None
    found_data = False
    nearest_station_info = None

    # iterate through nearby stations to find one with data
    for index, station in merged_nearby_stations_df.iterrows():
        nearest_station_id = station['station_id']

        # parameters for the API request
        params = {
            'product': 'water_temperature',
            'application': 'NOS.COOPS.TAC.PHYSOCEAN',
            'begin_date': begin_date_str,
            'end_date': end_date_str,
            'station': nearest_station_id,
            'units': 'english',
            'time_zone': 'lst',
            'format': 'json',
        }

        try:
            response = requests.get(water_temp_url, params=params)
            if response.status_code == 200:
                water_temp_data = response.json()
                if water_temp_data and 'data' in water_temp_data and water_temp_data['data']:
                    water_temp_result = water_temp_data

                    nearest_station_info = {
                        "station_id": nearest_station_id,
                        "station_name": station.get('City, State', 'N/A'),
                        "distance_miles": round(station.get('distance_miles', float('inf')), 2),
                        "latitude": station.get('Latitude', 'N/A'),
                        "longitude": station.get('Longitude', 'N/A'),
                        "source_url": response.url
                    }
                    found_data = True
                    break
            else:
                print(f"Error fetching water temperature data from station {nearest_station_id}: Status code {response.status_code}")
                print(f"Response content:\n{response.text}")
        except requests.exceptions.RequestException as e:
            print(f"Error during water temperature data request for station {nearest_station_id}: {e}")

    if found_data:
        reformatted_water_temp = []
        if water_temp_result and 'data' in water_temp_result:
            for entry in water_temp_result['data']:
                reformatted_water_temp.append({
                    "time_LST": entry.get('t'),
                    "value": entry.get('v'), # do not change name
                    "quality_flags": entry.get('f'),
                    "quality_control": entry.get('q')
                })

        # structure json file
        output_data = {
            "user_location": {
                "latitude": latitude,
                "longitude": longitude,
                "date": datetime.now().strftime('%Y-%m-%d')
            },
            "water_temperature": {
                "station_info": nearest_station_info,
                "data": reformatted_water_temp
            },
        }

        # save json file
        output_filename = "user_data_noaa_coops_watertemp.json"
        try:
            with open(output_filename, 'w') as f:
                json.dump(output_data, f, indent=4)
            print(f"Water temperature data saved to {output_filename}")
        except Exception as e:
            print(f"Error saving water temperatures to JSON: {e}")
    else:
        print("No water temperatures found for nearby CO-OPS stations.")

Water temperature data saved to user_data_noaa_coops_watertemp.json


#### Air Temp, Water Temp, Winds, Rain, Water Height


In [ ]:
# meteorological observations from CO-OPS Sensor Observation Service (SOS)

import requests
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta
import json

sos_url = "https://opendap.co-ops.nos.noaa.gov/ioos-dif-sos/SOS"

observed_properties = ["air_temperature", "winds", "rain_fall", "water_surface_height_above_reference_datum", "sea_water_temperature"]

# use nearest stations from merged_nearby_stations_df
if 'merged_nearby_stations_df' in globals() and not merged_nearby_stations_df.empty:
    # first five stations
    stations_ids = merged_nearby_stations_df['station_id'].tolist()[:5]
else:
    print("Nearest CO-OPS stations data is not available. Cannot fetch meteorological data.")
    stations_ids = []

today = datetime.now()
begin_date_str = today.strftime('%Y-%m-%d')
end_date_str = today.strftime('%Y-%m-%d')

response_format = "csv"
structured_met_data = {prop: {} for prop in observed_properties}

# construct string
event_time_string = f"{begin_date_str}T00:00:00Z/{end_date_str}T23:59:59Z"

if stations_ids:
    # iterate through observed properties and stations to fetch data
    for prop in observed_properties:
        for station_id in stations_ids:
            offering_string = f"urn:ioos:station:NOAA.NOS.CO-OPS:{station_id}"

            api_url = (
                f"{sos_url}?service=SOS&request=GetObservation&version=1.0.0"
                f"&observedProperty={prop}"
                f"&offering={offering_string}"
                f"&eventTime={event_time_string}"
                f"&responseFormat=text/{response_format}"
            )

            try:
                response = requests.get(api_url)
                if response.status_code == 200:

                    # read content into a pandas df if format is CSV
                    if response_format.lower() == 'csv':
                        try:
                            df = pd.read_csv(StringIO(response.text), comment='#')

                            # store df in the new structured dictionary
                            structured_met_data[prop][station_id] = df.to_dict(orient='records')

                        except Exception as e:
                            print(f"Error parsing CSV data for {prop} at station {station_id}: {e}")
                            print("Response content head:")
                            print(response.text[:500])
                    else:
                        print(f"Fetched data for {prop} at station {station_id} (non-CSV format):")
                        print(response.text[:500])
                else:
                    print(f"Error fetching data for {prop} at station {station_id}: Status code {response.status_code}")
                    print(f"Response content:\n{response.text}")
            except requests.exceptions.RequestException as e:
                print(f"Error during data request for {prop} at station {station_id}: {e}")
else:
    print("No stations available to fetch meteorological data.")

# save to json file
if any(structured_met_data[prop] for prop in observed_properties):
    output_filename = "user_data_noaa_sos.json"

    # structure json file
    output_data = {
        "description": {
            "about": "The meteorological observations provided comes from the National Oceanic and Atmospheric Administration's (NOAA), Center for Operational Oceanographic Products and Services (CO-OPS), Sensor Observation Service (SOS). Learn more through # https://opendap.co-ops.nos.noaa.gov/ioos-dif-sos/",
            "parameters": "air temperature, winds, rain fall, water surface height (above reference datum), surface water temperature."
        },
        "meteorological_data": structured_met_data
    }

    try:
        with open(output_filename, 'w') as f:
            json.dump(output_data, f, indent=4)
        print(f"Meteorological data saved to {output_filename}")
    except Exception as e:
        print(f"Error saving meteorological data to JSON: {e}")
else:
    print("No meteorological data fetched to save.")

Meteorological data saved to user_data_noaa_sos.json


### Atmospheric Visualizations

#### Hourly Graphs

In [ ]:
# hourly temperature forecast line graph -------------------------------------

%pip install matplotlib

import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates
from datetime import date

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        temperatures = [period['temperature'] for period in hourly_periods_today]

        plt.figure(figsize=(10, 6))
        plt.plot(times, temperatures, marker='o')
        plt.xlabel("Time")
        plt.ylabel(f"Temperature ({hourly_periods_today[0]['temperatureUnit']})")
        plt.title(f"Hourly Temperature Forecast for {today.strftime('%Y-%m-%d')}")

        # format x-axis to show time with appropriate intervals
        ax = plt.gca()
        formatter = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(formatter)

        # format tick locations
        import numpy as np
        hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
        ax.xaxis.set_major_locator(hours)

        plt.gcf().autofmt_xdate() # auto-format to prevent label overlapping

        plt.tight_layout()
        plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly apparent temperature forecast line graph ----------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    today = date.today() # filter for the current date
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]

        # use apparent temperature if available, otherwise use temperature
        temperatures = []
        for period in hourly_periods_today:
            apparent_temp = period.get('apparentTemperature', {}).get('value')
            if apparent_temp is not None:
                temperatures.append(apparent_temp)
            else:
                temperatures.append(period.get('temperature'))

        # determine the unit
        temp_unit = hourly_periods_today[0].get('temperatureUnit', '')

        plt.figure(figsize=(10, 6))
        plt.plot(times, temperatures, marker='o')
        plt.xlabel("Time")
        plt.ylabel(f"Temperature ({temp_unit})") # label reflects whats plotted
        plt.title(f"Hourly Temperature (Apparent if available) Forecast for {today.strftime('%Y-%m-%d')}")

        # format x-axis to show time with appropriate intervals
        ax = plt.gca()
        formatter = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(formatter)

        # format tick locations
        import numpy as np
        hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
        ax.xaxis.set_major_locator(hours)

        plt.gcf().autofmt_xdate() # auto-format to prevent label overlapping

        plt.tight_layout()
        plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly cloud cover forecast line graph -------------------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # extract cloud cover - need to handle potential none values
        cloud_cover_values = [period.get('cloudCover', {}).get('value') for period in hourly_periods_today]

        # remove none values for plotting
        times_filtered = [times[i] for i, val in enumerate(cloud_cover_values) if val is not None]
        cloud_cover_values_filtered = [val for val in cloud_cover_values if val is not None]


        if not cloud_cover_values_filtered:
             print("Cloud Cover data not available for visualization for the current date.")
        else:
            plt.figure(figsize=(10, 6))
            plt.plot(times_filtered, cloud_cover_values_filtered, marker='o')
            plt.xlabel("Time")
            plt.ylabel("Cloud Cover (%)")
            plt.title(f"Hourly Cloud Cover Forecast for {today.strftime('%Y-%m-%d')}")

            # format x-axis to show only time and set appropriate intervals
            ax = plt.gca()
            formatter = mdates.DateFormatter('%H:%M')
            ax.xaxis.set_major_formatter(formatter)

            import numpy as np
            hours = mdates.HourLocator(interval=3) # set a tick every 3 hours
            ax.xaxis.set_major_locator(hours)


            plt.gcf().autofmt_xdate() # format to prevent label overlapping

            plt.tight_layout()
            plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly precipitation forecast probability line graph -----------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # extract probability of precipitation
        precipitation_probs = [period.get('probabilityOfPrecipitation', {}).get('value', 0) for period in hourly_periods_today]

        plt.figure(figsize=(10, 6))
        plt.plot(times, precipitation_probs, marker='o')
        plt.xlabel("Time")
        plt.ylabel("Probability of Precipitation (%)")
        plt.title(f"Hourly Probability of Precipitation Forecast for {today.strftime('%Y-%m-%d')}")

        # format x-axis to show only time and set appropriate intervals
        ax = plt.gca()
        formatter = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(formatter)

        import numpy as np
        hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
        ax.xaxis.set_major_locator(hours)


        plt.gcf().autofmt_xdate() # auto-format to prevent labels overlapping

        plt.tight_layout()
        plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly humidity forecast line graph ----------------------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # Extract relative humidity
        humidity_values = [period.get('relativeHumidity', {}).get('value', 0) for period in hourly_periods_today]

        plt.figure(figsize=(10, 6))
        plt.plot(times, humidity_values, marker='o')
        plt.xlabel("Time")
        plt.ylabel("Relative Humidity (%)")
        plt.title(f"Hourly Relative Humidity Forecast for {today.strftime('%Y-%m-%d')}")

        # format the x-axis to show only time and set appropriate intervals
        ax = plt.gca()
        formatter = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(formatter)

        import numpy as np
        hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
        ax.xaxis.set_major_locator(hours)


        plt.gcf().autofmt_xdate() # auto-format to prevent labels overlapping

        plt.tight_layout()
        plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly visibility forecast line graph --------------------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # extract visibility
        visibility_values = []
        visibility_unit = '' # initialize unit
        for period in hourly_periods_today:
            visibility_data = period.get('visibility', {})
            value = visibility_data.get('value')
            if value is not None:
                visibility_values.append(value)
                # capture the unit from the first available data point
                if not visibility_unit and visibility_data.get('unitCode'):
                    visibility_unit = visibility_data.get('unitCode')
            else:
                visibility_values.append(None) # append if data is missing

        # remove None values for plotting
        times_filtered = [times[i] for i, val in enumerate(visibility_values) if val is not None]
        visibility_values_filtered = [val for val in visibility_values if val is not None]


        if not visibility_values_filtered:
             print("Visibility data not available for visualization for the current date.")
        else:
            plt.figure(figsize=(10, 6))
            plt.plot(times_filtered, visibility_values_filtered, marker='o')
            plt.xlabel("Time")
            # set ylabel based on the captured unit
            ylabel = "Visibility"
            if visibility_unit:
                 ylabel += f" ({visibility_unit})"

            plt.ylabel(ylabel)
            plt.title(f"Hourly Visibility Forecast for {today.strftime('%Y-%m-%d')}")

            # format x-axis to show only time and set appropriate intervals
            ax = plt.gca()
            formatter = mdates.DateFormatter('%H:%M')
            ax.xaxis.set_major_formatter(formatter)

            import numpy as np
            hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
            ax.xaxis.set_major_locator(hours)

            plt.gcf().autofmt_xdate() # auto-format for labels overlapping

            plt.tight_layout()
            plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly dewpoint forecast line graph ----------------------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # extract dewpoint - need to handle potential None values
        dewpoint_values = [period.get('dewpoint', {}).get('value') for period in hourly_periods_today]

        # convert dewpoint from C to F if temperature unit is F
        if hourly_periods_today and hourly_periods_today[0].get('temperatureUnit') == 'F':
             dewpoint_values = [(d * 9/5) + 32 if d is not None else None for d in dewpoint_values]
             dewpoint_unit = 'F'
        elif hourly_periods_today and hourly_periods_today[0].get('dewpoint') and hourly_periods_today[0]['dewpoint'].get('unitCode'):
             dewpoint_unit = hourly_periods_today[0]['dewpoint'].get('unitCode')
        else:
             dewpoint_unit = ''


        # remove None values for plotting
        times_filtered = [times[i] for i, val in enumerate(dewpoint_values) if val is not None]
        dewpoint_values_filtered = [val for val in dewpoint_values if val is not None]


        if not dewpoint_values_filtered:
             print("Dewpoint data not available for visualization for the current date.")
        else:
            plt.figure(figsize=(10, 6))
            plt.plot(times_filtered, dewpoint_values_filtered, marker='o')
            plt.xlabel("Time")
            plt.ylabel(f"Dewpoint (°{dewpoint_unit})")
            plt.title(f"Hourly Dewpoint Forecast for {today.strftime('%Y-%m-%d')}")

            # format x-axis to show only time and set appropriate intervals
            ax = plt.gca()
            formatter = mdates.DateFormatter('%H:%M')
            ax.xaxis.set_major_formatter(formatter)

            import numpy as np
            hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
            ax.xaxis.set_major_locator(hours)


            plt.gcf().autofmt_xdate() # format to prevent label overlapping

            plt.tight_layout()
            plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly wind speed forecast line graph --------------------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # extract wind speed
        wind_speeds = []
        for period in hourly_periods_today:
            # for cases when windSpeed is 0
            speed_str = period.get('windSpeed', '0 mph').split(' ')[0]
            try:
                wind_speeds.append(int(speed_str))
            except ValueError:
                # for cases when windSpeed is a range; takes the first number
                 try:
                     wind_speeds.append(int(speed_str.split(' to ')[0]))
                 except ValueError:
                     wind_speeds.append(0) # default to 0 if parsing fails

        plt.figure(figsize=(10, 6))
        plt.plot(times, wind_speeds, marker='o')
        plt.xlabel("Time")
        plt.ylabel("Wind Speed (mph)")
        plt.title(f"Hourly Wind Speed Forecast for {today.strftime('%Y-%m-%d')}")

        # format x-axis to show only time and set appropriate intervals
        ax = plt.gca()
        formatter = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(formatter)

        import numpy as np
        hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
        ax.xaxis.set_major_locator(hours)


        plt.gcf().autofmt_xdate() # auto-format to prevent labels overlapping

        plt.tight_layout()
        plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly wind gust forecast line graph ---------------------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # extract windGust
        wind_gusts = []
        for period in hourly_periods_today:
            gust_str = period.get('windGust')
            if gust_str: # check if windGust exists and is not None/empty
                try:
                    # for when windGust is a number or a range
                    if ' to ' in gust_str:
                        wind_gusts.append(int(gust_str.split(' to ')[0]))
                    else:
                        # assume unit is separated by space
                        wind_gusts.append(int(gust_str.split(' ')[0]))
                except (ValueError, IndexError):
                    wind_gusts.append(0) # default to 0 if parsing fails
            else:
                wind_gusts.append(0) # append 0 if windGust is missing


        plt.figure(figsize=(10, 6))
        plt.plot(times, wind_gusts, marker='o')
        plt.xlabel("Time")
        plt.ylabel("Wind Gust (mph)") # changed label to Wind Gust
        plt.title(f"Hourly Wind Gust Forecast for {today.strftime('%Y-%m-%d')}") # changed title

        # set the y-axis limit to start at 0
        plt.ylim(bottom=0)

        # format x-axis to show only time and set appropriate intervals
        ax = plt.gca()
        formatter = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(formatter)

        import numpy as np
        hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
        ax.xaxis.set_major_locator(hours)


        plt.gcf().autofmt_xdate() # auto-format to prevent labels overlapping

        plt.tight_layout()
        plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly sky cover forecast line graph --------------------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # extract skyCover - need to handle potential None values
        sky_cover_values = [period.get('skyCover', {}).get('value') for period in hourly_periods_today]

        # remove None values for plotting
        times_filtered = [times[i] for i, val in enumerate(sky_cover_values) if val is not None]
        sky_cover_values_filtered = [val for val in sky_cover_values if val is not None]

        if not sky_cover_values_filtered:
             print("Sky Cover data not available for visualization for the current date.")
        else:
            plt.figure(figsize=(10, 6))
            plt.plot(times_filtered, sky_cover_values_filtered, marker='o')
            plt.xlabel("Time")
            plt.ylabel("Sky Cover (%)")
            plt.title(f"Hourly Sky Cover Forecast for {today.strftime('%Y-%m-%d')}")

            # format x-axis to show only time and set appropriate intervals
            ax = plt.gca()
            formatter = mdates.DateFormatter('%H:%M')
            ax.xaxis.set_major_formatter(formatter)

            import numpy as np
            hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
            ax.xaxis.set_major_locator(hours)


            plt.gcf().autofmt_xdate() # format to prevent label overlapping

            plt.tight_layout()
            plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly heat / UV index forecast line graph ---------------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # extract heat index - need to handle potential None values
        heat_index_values = [period.get('heatIndex', {}).get('value') for period in hourly_periods_today]

        # determine the unit if available
        heat_index_unit = ''
        if hourly_periods_today and hourly_periods_today[0].get('heatIndex') and hourly_periods_today[0]['heatIndex'].get('unitCode'):
             heat_index_unit = hourly_periods_today[0]['heatIndex'].get('unitCode')


        # remove None values for plotting
        times_filtered = [times[i] for i, val in enumerate(heat_index_values) if val is not None]
        heat_index_values_filtered = [val for val in heat_index_values if val is not None]


        if not heat_index_values_filtered:
             print("Heat Index data not available for visualization for the current date.")
        else:
            plt.figure(figsize=(10, 6))
            plt.plot(times_filtered, heat_index_values_filtered, marker='o')
            plt.xlabel("Time")
            plt.ylabel(f"Heat Index (°{heat_index_unit})")
            plt.title(f"Hourly Heat Index Forecast for {today.strftime('%Y-%m-%d')}")

            # format x-axis to show only time and set appropriate intervals
            ax = plt.gca()
            formatter = mdates.DateFormatter('%H:%M')
            ax.xaxis.set_major_formatter(formatter)

            import numpy as np
            hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
            ax.xaxis.set_major_locator(hours)


            plt.gcf().autofmt_xdate() # format to prevent label overlapping

            plt.tight_layout()
            plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly snowfall amount forecast line graph ---------------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # extract snowfallAmount
        snowfall_values = []
        snowfall_unit = ''
        for period in hourly_periods_today:
            snowfall_data = period.get('snowfallAmount', {})
            value = snowfall_data.get('value')
            if value is not None:
                snowfall_values.append(value)
                # capture the unit from the first available data point
                if not snowfall_unit and snowfall_data.get('unitCode'):
                    snowfall_unit = snowfall_data.get('unitCode')
            else:
                snowfall_values.append(0) # append 0 if data is missing


        if not any(snowfall_values): # check for any snowfall predictions
             print("Snowfall data not available or no snowfall predicted for the current date.")
        else:
            plt.figure(figsize=(10, 6))
            plt.plot(times, snowfall_values, marker='o')
            plt.xlabel("Time")
            plt.ylabel(f"Snowfall Amount ({snowfall_unit})")
            plt.title(f"Hourly Snowfall Amount Forecast for {today.strftime('%Y-%m-%d')}")

            # format x-axis to show only time and set appropriate intervals
            ax = plt.gca()
            formatter = mdates.DateFormatter('%H:%M')
            ax.xaxis.set_major_formatter(formatter)

            import numpy as np
            hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
            ax.xaxis.set_major_locator(hours)


            plt.gcf().autofmt_xdate() # format to prevent label overlapping

            plt.tight_layout()
            plt.show()
else:
    print("Hourly forecast data not available for visualization.")



# hourly ice accumulation forecast line graph --------------------------------

if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # filter for the current date
    today = date.today()
    hourly_periods_today = [
        period for period in hourly_periods
        if datetime.fromisoformat(period['startTime']).date() == today
    ]

    if not hourly_periods_today:
        print("No hourly forecast data available for the current date.")
    else:
        times = [datetime.fromisoformat(period['startTime']) for period in hourly_periods_today]
        # extract iceAccumulation
        ice_accumulation_values = []
        ice_accumulation_unit = ''
        for period in hourly_periods_today:
            ice_data = period.get('iceAccumulation', {})
            value = ice_data.get('value')
            if value is not None:
                ice_accumulation_values.append(value)
                # capture the unit from the first available data point
                if not ice_accumulation_unit and ice_data.get('unitCode'):
                    ice_accumulation_unit = ice_data.get('unitCode')
            else:
                # append 0 if data is missing
                ice_accumulation_values.append(0)

        # check if there is any ice accumulation predicted
        if not any(ice_accumulation_values):
             print("Ice accumulation data not available or no ice accumulation predicted for the current date.")
        else:
            plt.figure(figsize=(10, 6))
            plt.plot(times, ice_accumulation_values, marker='o')
            plt.xlabel("Time")
            plt.ylabel(f"Ice Accumulation ({ice_accumulation_unit})")
            plt.title(f"Hourly Ice Accumulation Forecast for {today.strftime('%Y-%m-%d')}")

            # format x-axis to show only time and set appropriate intervals
            ax = plt.gca()
            formatter = mdates.DateFormatter('%H:%M')
            ax.xaxis.set_major_formatter(formatter)

            import numpy as np
            hours = mdates.HourLocator(interval=3)  # set a tick every 3 hours
            ax.xaxis.set_major_locator(hours)


            plt.gcf().autofmt_xdate() # format to prevent label overlapping

            plt.tight_layout()
            plt.show()
else:
    print("Hourly forecast data not available for visualization.")

#### Hourly Container +7-Day

In [ ]:
# hourly forecast container for 7-days

import json
from datetime import datetime # Import datetime for parsing the string timestamps


if weather_data_result and 'hourly_forecast' in weather_data_result:
    hourly_periods = weather_data_result['hourly_forecast']['properties']['periods']

    # convert datetime objects to strings for JSON serialization
    # for period in hourly_periods:
    #     period['startTime'] = period['startTime'].isoformat()
    #     period['endTime'] = period['endTime'].isoformat()

    print("\n--- Hourly Forecast ---")
    for period in hourly_periods:
        # Parse the string timestamps into datetime objects for printing
        start_time_dt = datetime.fromisoformat(period['startTime'])
        print(f"\nTime: {start_time_dt.strftime('%Y-%m-%d %H:%M')}")
        print(f"  Temperature: {period.get('temperature')}°{period.get('temperatureUnit', '')}")

        if period.get('windChill') and period['windChill'].get('value') is not None:
             print(f"  Wind Chill: {period['windChill']['value']}°{period['windChill'].get('unitCode', '')}")

        if period.get('heatIndex') and period['heatIndex'].get('value') is not None:
             print(f"  Heat Index: {period['heatIndex']['value']}°{period['heatIndex'].get('unitCode', '')}")
        print(f"  Wind Speed: {period.get('windSpeed', 'N/A')}")
        print(f"  Wind Direction: {period.get('windDirection', 'N/A')}")

        if period.get('cloudCover') and period['cloudCover'].get('value') is not None:
             print(f"  Cloud Cover: {period['cloudCover']['value']}%")

        if period.get('probabilityOfPrecipitation') and period['probabilityOfPrecipitation'].get('value') is not None:
             print(f"  Probability of Precipitation: {period['probabilityOfPrecipitation']['value']}%")

        if period.get('dewpoint') and period['dewpoint'].get('value') is not None:
             # convert dewpoint from C to F if temperature unit is F
             dewpoint_value = period['dewpoint']['value']
             dewpoint_unit = period['dewpoint'].get('unitCode', '')
             if period.get('temperatureUnit') == 'F' and dewpoint_unit == 'wmoUnit:degC':
                 dewpoint_value = (dewpoint_value * 9/5) + 32
                 dewpoint_unit = 'F' # update unit to F

             # if the original unit is already F (unlikely)
             elif period.get('temperatureUnit') == 'F' and dewpoint_unit == 'wmoUnit:degF':
                 dewpoint_unit = 'F'

             print(f"  Dewpoint: {dewpoint_value:.0f}°{dewpoint_unit}")


        if period.get('relativeHumidity') and period['relativeHumidity'].get('value') is not None:
             print(f"  Relative Humidity: {period['relativeHumidity']['value']}%")

        print(f"  Short Forecast: {period.get('shortForecast', 'N/A')}")


    # save hourly forecast data to JSON file
    hourly_output_filename = "noaa_hourly_7day_forecast.json"
    try:
        with open(hourly_output_filename, 'w') as f:
            json.dump(hourly_periods, f, indent=4)
        print(f"\nHourly forecast data saved to {hourly_output_filename}")
    except Exception as e:
        print(f"\nError saving hourly forecast data to JSON: {e}")

else:
    print("Hourly forecast data not available.")

#### 7-Day Graphs

In [ ]:
# 7-day temperature bar graph ------------------------------------------------

import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates

if weather_data_result and 'forecast' in weather_data_result:
    forecast_periods = weather_data_result['forecast']['properties']['periods']

    if not forecast_periods:
        print("7-day forecast data not available for visualization.")
    else:
        # extract times, temperatures, and daytime status for each period
        times = [datetime.fromisoformat(period['startTime']) for period in forecast_periods]
        temperatures = [period['temperature'] for period in forecast_periods]
        is_daytime = [period['isDaytime'] for period in forecast_periods]

        # eetermine the unit
        temp_unit = forecast_periods[0].get('temperatureUnit', '')

        # define colors for day and night
        colors = ['skyblue' if day else 'darkblue' for day in is_daytime]

        plt.figure(figsize=(10, 6))
        # plot bars with different colors based on daytime status
        plt.bar(times, temperatures, width=0.4, color=colors)
        plt.xlabel("Day")
        plt.ylabel(f"Temperature ({temp_unit})")
        plt.title("7-Day Temperature Forecast")

        # format the x-axis to show days of the week
        ax = plt.gca()
        formatter = mdates.DateFormatter('%A')
        ax.xaxis.set_major_formatter(formatter)

        # set tick locations (one tick per day)
        ax.xaxis.set_major_locator(mdates.DayLocator())


        plt.gcf().autofmt_xdate() # auto-format to prevent labels overlapping

        plt.tight_layout()
        plt.show()
else:
    print("7-day forecast data not available for visualization.")


# 7-day precipitation probability line graph ---------------------------------

import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates

if weather_data_result and 'forecast' in weather_data_result:
    forecast_periods = weather_data_result['forecast']['properties']['periods']

    if not forecast_periods:
        print("7-day forecast data not available for visualization.")
    else:
        # extract times and probability of precipitation for each period
        times = [datetime.fromisoformat(period['startTime']) for period in forecast_periods]
        # extract probability of precipitation, handling potential None values
        precipitation_probs = [period.get('probabilityOfPrecipitation', {}).get('value', 0) for period in forecast_periods]

        plt.figure(figsize=(10, 6))
        plt.plot(times, precipitation_probs, marker='o')
        plt.xlabel("Day")
        plt.ylabel("Probability of Precipitation (%)")
        plt.title("7-Day Probability of Precipitation Forecast")

        # format the x-axis to show days of the week
        ax = plt.gca()
        formatter = mdates.DateFormatter('%A')
        ax.xaxis.set_major_formatter(formatter)

        # set tick locations (one tick per day)
        ax.xaxis.set_major_locator(mdates.DayLocator())


        plt.gcf().autofmt_xdate() # auto-format to prevent labels overlapping

        plt.tight_layout()
        plt.show()
else:
    print("7-day forecast data not available for visualization.")

#### 7-Day Container +Alerts

In [ ]:
# 7-day forecast container

import json
from datetime import datetime, timezone

if weather_data_result and 'forecast' in weather_data_result:
    forecast_periods = weather_data_result['forecast']['properties']['periods']
    print("--- 7-Day Forecast ---")

    # Print the total number of active weather alerts found
    if parsed_alerts:
        print(f"Total active weather alerts found for {user_location}: {len(parsed_alerts)}")
    else:
        print(f"No active weather alerts found for {user_location}.")

    for period in forecast_periods:
        # convert period times to datetime objects (assuming ISO 8601 format)
        start_time = datetime.fromisoformat(period['startTime'])
        end_time = datetime.fromisoformat(period['endTime'])

        print(f"\n{period['name']}:")
        print(f"  Temperature: {period.get('temperature')}°{period.get('temperatureUnit', '')}")
        print(f"  Short Forecast: {period.get('shortForecast', 'N/A')}")
        print(f"  Detailed Forecast: {period.get('detailedForecast', 'N/A')}")
        print(f"  Wind Speed: {period.get('windSpeed', 'N/A')}")
        print(f"  Wind Direction: {period.get('windDirection', 'N/A')}")

        # include probability of precipitation
        if period.get('probabilityOfPrecipitation') and period['probabilityOfPrecipitation'].get('value') is not None:
             print(f"  Probability of Precipitation: {period['probabilityOfPrecipitation']['value']}%")

        # check for relevant active alerts within forecast period
        relevant_alerts = []
        if parsed_alerts:
            for alert in parsed_alerts:
                alert_effective = datetime.fromisoformat(alert.get('effective')) if alert.get('effective') else None
                alert_expires = datetime.fromisoformat(alert.get('expires')) if alert.get('expires') else None

                # check if alert times overlap with the forecast period
                if alert_effective and alert_expires:
                    # an alert is relevant if it starts before or during the period and ends after or during the period
                    if (alert_effective <= end_time and alert_expires >= start_time):
                         relevant_alerts.append(alert)
                # alert with no explicit end time
                elif alert_effective and not alert_expires:
                     if alert_effective <= end_time:
                          relevant_alerts.append(alert)
                # alert with no explicit start time (uncommon)
                elif not alert_effective and alert_expires:
                     if alert_expires >= start_time:
                           relevant_alerts.append(alert)
                # alert with no effective or expires time
                else:
                     pass


        if relevant_alerts:
            print("  Active Alerts:")
            for rel_alert in relevant_alerts:
                print(f"    - Event: {rel_alert.get('event', 'N/A')}")
                print(f"      Headline: {rel_alert.get('headline', 'N/A')}")

    # save 7-day forecast data to JSON file
    forecast_output_filename = "noaa_7day_forecast.json"
    try:
        with open(forecast_output_filename, 'w') as f:
            json.dump(forecast_periods, f, indent=4)
        print(f"\n7-day forecast data saved to {forecast_output_filename}")
    except Exception as e:
        print(f"\nError saving 7-day forecast data to JSON: {e}")

else:
    print("Forecast data not available.")

#### Alerts Container


In [ ]:
# active weather alerts container

# iterate through the parsed alerts
if parsed_alerts:
    print(f"--- Active Weather Alerts for {user_location} ---")
    for alert in parsed_alerts:
        print(f"\nEvent: {alert.get('event', 'N/A')}")
        print(f"Headline: {alert.get('headline', 'N/A')}")
        print(f"Description: {alert.get('description', 'N/A')}")
        print(f"Effective: {alert.get('effective', 'N/A')}")
        print(f"Expires: {alert.get('expires', 'N/A')}")
        print("-" * 20) # separator for clarity
else:
    print(f"No active weather alerts found for {user_location}.")

#### Air Quality Graph

In [ ]:
# AQ (air quality) graph

import matplotlib.pyplot as plt
import pandas as pd

if aq_data:
    # convert list of dictionaries to a pandas DataFrame for easier handling
    aq_df = pd.DataFrame(aq_data)

    if not aq_df.empty:
      # extract date from the first entry (assuming all entries have same date)
      observation_date = aq_df['DateObserved'].iloc[0] if 'DateObserved' in aq_df.columns else "Unknown Date"

      # create a bar chart for AQI by Parameter
      plt.figure(figsize=(10, 6))
      plt.bar(aq_df['ParameterName'], aq_df['AQI'], color=['skyblue', 'lightgreen', 'salmon'])
      plt.xlabel("Parameter")
      plt.ylabel("AQI (Air Quality Index)")
      # add the date to the title
      plt.title(f"Air Quality Index by Parameter on {observation_date}")
      plt.ylim(0, aq_df['AQI'].max() * 1.2) # set y-axis limit a bit above the max AQI
      plt.tight_layout()
      plt.show()
    else:
      print("Air quality data is empty, cannot create visualizations.")
else:
  print("Air quality data not available for visualization.")

#### Air Quality Container

In [ ]:
# air quality container

if aq_data:
    print("--- Air Quality Data ---")
    for entry in aq_data:
        print(f"\nDate Observed: {entry.get('DateObserved', 'N/A')}")
        print(f"Hour Observed: {entry.get('HourObserved', 'N/A')} {entry.get('LocalTimeZone', 'N/A')}")
        print(f"Reporting Area: {entry.get('ReportingArea', 'N/A')}")
        print(f"Parameter Name: {entry.get('ParameterName', 'N/A')}")
        print(f"AQI: {entry.get('AQI', 'N/A')}")
        # access the Name from the Category dictionary
        category_name = entry.get('Category', {}).get('Name', 'N/A')
        print(f"Category: {category_name}")
        print("-" * 20) # separator for clarity
else:
    print("Air quality data not available.")